# Optimisation via Simulation Tutorial

By Christine S.M Currie and Tom Monks

In [1]:
import numpy as np

## Models

To do.

* Need to rename some of the functions so that it fits
* Need to adapt Christine's Law inventory model

In [2]:
from ovs.toy_models import (custom_guass_bandit_problem,
                            BanditCasino, 
                            guassian_bandit_sequence)

## Procedure **KN**

To run Kim and Nelson's R&S procedure KN, create an instance of `ovs.indifference_zone.KN`

An object of type KN takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **delta** - float, the indifference zone
* **alpha** - float, $PCS = 1-\alpha$ (default=0.05)
* **n_0** - int, $n_0$ the number of initial replications (default=2)

In [3]:
from ovs.indifference_zone import KN

In [4]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.05, 
        n_0=5)

In [5]:
best_design = kn.solve()
print('best design\t{0}'.format(best_design))
print('allocations\t{0}'.format(kn._actions))
print('total reps\t{0}'.format(kn._actions.sum()))
print('means\t\t{0}'.format(kn._means))

best design	[9]
allocations	[143  64   5  39  34  35  36 118 271 272]
total reps	1017
means		[0.96503169 1.73607296 3.61972753 4.10576491 5.18083282 6.20369728
 7.24688036 8.12468075 9.02511827 9.99795459]


## Optimal Computing Budget Allocation (OCBA)

An object of type OCBA takes the following parameters:

* **model** - a simulation model
* **n_designs** - int, the number of competing designs to compare
* **budget** - int, the total number of replications to allocate across designs
* **delta** - int, the incremental amount of replications to allocate at each round
* **n_0** - int, $n_0$ the number of initial replications (default=5)
* **min** - bool, True if minimisation; False if maximisation (default=False)

In [6]:
from ovs.fixed_budget import OCBA

In [7]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=300, 
            delta=10, 
            n_0=5, 
            min=True)

call the `solve()` method to run the optimisation

In [8]:
results = ocba.solve()
print('best design:\t{}'.format(results))
print('allocations:\t{}'.format(ocba._allocations))
print('total reps:\t{}'.format(ocba._allocations.sum()))

np.set_printoptions(precision=2)
print('means:\t\t{0}'.format(ocba._means))
print('vars:\t\t{0}'.format(ocba._vars))

best design:	0
allocations:	[246  14   5   5   5   5   5   5   5   5]
total reps:	300
means:		[ 1.02  1.9   3.59  3.55  4.27  5.88  7.33  7.85  8.93 10.13]
vars:		[1.13 0.7  1.14 0.37 0.72 0.62 0.8  1.8  2.21 0.92]


/home/tom/Dropbox/Python/ovs-tutorial/ovs/fixed_budget.py:206: RuntimeWarning: divide by zero encountered in true_divide
  (budget_to_allocate / ratio_s * self._ratios[more_runs])


# Evaluation of the methods

In [9]:
from ovs.evaluation import Experiment

In [12]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

kn = KN(model=environment, 
        n_designs=len(designs), 
        delta=0.1, 
        alpha=0.2, 
        n_0=5)

exp = Experiment(env=environment, procedure=kn, best_index=9, replications=10)

In [13]:
results = exp.execute()

In [14]:
results.p_correct_selections

1.0

In [15]:
results.selections

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=int32)

In [16]:
results.correct_selections

10

In [82]:
designs = guassian_bandit_sequence(1, 11)
environment = BanditCasino(designs)

ocba = OCBA(model=environment, 
            n_designs=len(designs), 
            budget=200, 
            delta=10, 
            n_0=5, 
            min=False)

exp = Experiment(env=environment, procedure=ocba, best_index=9, replications=50)

In [83]:
results = exp.execute()

In [84]:
results.p_correct_selections

1.0

In [85]:
results.selections

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9], dtype=int32)

In [86]:
results.correct_selections

50

## Try different budgets

In [88]:
from ovs.evaluation import GridExperiment

In [91]:
param_grid = {'budget':[100, 200, 300, 400, 500], 
              'n_designs':len(designs),
              'delta':[5,10],
              'n_0':5,
              'min':False}


exp = GridExperiment(agent=ocba, 
                     environment=environment, 
                     param_grid=param_grid,
                     best_index=9,
                     replications=10)

TypeError: Parameter grid value is not iterable (key='n_designs', value=10)